# プロパティ付きのトリプルを使って Graph DB 作る

## 事前準備

Neo4jを立ち上げておく

```bash
cd {project_home}
docker compose up -d
```

ブラウザで [http://localhost:7474/]( http://localhost:7474/) でWebインターフェースに入れる。

## 実装

In [1]:
import json
import os
from pathlib import Path

from dotenv import load_dotenv
from neo4j import GraphDatabase

In [2]:
load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "")

print(f"URI: {NEO4J_URI}")
print(f"USER: {NEO4J_USER}")

URI: bolt://localhost:7687
USER: neo4j


### データロード

テスト用に、LLMで作ったプロパティ付きのトリプルをJSONファイルにしてあるので、これを利用する

In [3]:
def load_triples_from_file(path: Path) -> list[dict]:
    with path.open(encoding="utf-8") as f:
        return json.load(f)

In [4]:
TRIPLES_JSON = Path("../data/phase_4_outputs/triples_llm_with_properties.json")

In [5]:
triples = load_triples_from_file(TRIPLES_JSON)
print(len(triples))

6


In [6]:
triples[0]

{'subject': {'name': 'logistic_regression', 'type': 'model'},
 'object': {'name': 'model', 'type': 'concept'},
 'relation': {'type': 'BE',
  'verb': 'be',
  'source_text': 'Logistic Regression is a model widely used for binary classification.',
  'extractor': 'llm',
  'confidence': 0.95,
  'doc_id': 'ml_intro.txt',
  'sentence_id': 0}}

### データをinsertする

insert する前に、古いデータが残っているなら削除しといてもよい

Web I/F で下記実行

```
MATCH (n)
DETACH DELETE n;
```

In [7]:
def insert_triples(triples: list[dict]) -> None:
    """List[dict] のトリプルを Neo4j に投入する"""

    driver = GraphDatabase.driver(
        NEO4J_URI,
        auth=(NEO4J_USER, NEO4J_PASSWORD)
    )

    query = """
    UNWIND $rows AS row
    
    MERGE (s:Entity {
        name: row.subject.name,
        type: row.subject.type
    })
    MERGE (o:Entity {
        name: row.object.name,
        type: row.object.type
    })

    MERGE (s)-[r:REL {verb: row.relation.type}]->(o)
    SET
        r.source_text = row.relation.source_text,
        r.confidence = row.relation.confidence,
        r.sentence_id = row.relation.sentence_id
    """

    with driver.session() as session:
        session.run(query, rows=triples)

    driver.close()

In [8]:
insert_triples(triples)